# RAG

Retrieval-Augmented Generation (RAG) is a technique that combines the strengths of pre-trained language models with the ability to retrieve information from a large corpus of documents. RAG **enables the language model to produce more informed, accurate, and contextually relevant answers** than by relying on its pre-trained knowledge alone.

At Cohere, all RAG calls come with... **precise citations**! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from and they help users gain visibility into the model reasoning.  

RAG consists of 3 steps:
- Step 1: Indexing and given a user query, retrieve the relevant chunks from the index
- Step 2: Optionally, rerank the retrieved chunks
- Step 3: Generate the model final answer with **precise citations**, given the retrieved and reranked chunks




## Step 0 - Imports & Getting some data

In this example, we'll use a recent piece of text, that wasn't in the training data: the Wikipedia page of the movie "Dune 2".   

In practice, you would typically do RAG on much longer text, that doesn't fit in the context window of the model.

In [ ]:
# we'll use Cohere to cover all building blocks of RAG
%pip install cohere wikipedia langchain-text-splitters numpy --quiet

In [ ]:
import cohere
API_KEY = "..." # fill in your Cohere API key here
co = cohere.ClientV2(API_KEY)

In [ ]:
# we'll get some wikipedia data
import wikipedia

  Preparing metadata (setup.py) ... done


In [3]:
# let's get the wikipedia article about Dune Part Two
article = wikipedia.page('Dune Part Two')
text = article.content
print(f"The text has roughly {len(text.split())} words.")

The text has roughly 6176 words.


## Step 1 - Indexing and given a user query, retrieve the relevant chunks from the index

We index the document in a vector database. This requires getting the documents, chunking them, embedding, and indexing them in a vector database. Then we retrieved relevant results based on the users' query.

### We split the document into chunks of roughly 512 words

In [9]:
# For chunking let's use langchain to help us split the text
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents([text])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 62 chunks.


### Embed every text chunk

Cohere embeddings are state-of-the-art.

In [27]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model="embed-v4.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 62 embeddings.


### Store the embeddings in a vector database

We use the simplest vector database ever: a python dictionary using `np.array()`.

In [28]:
import numpy as np
vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}
# { 0: array([...]), 1: array([...]), 2: array([...]), ..., 10: array([...]) }

## Given a user query, retrieve the relevant chunks from the vector database


### Define the user question

In [29]:
query = "Name everyone involved in writing the script, directing, and producing 'Dune: Part Two'?"

# Note: the relevant passage in the wikipedia page we're looking for is:
# "[...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

### Embed the user question

Cohere embeddings are state-of-the-art.

In [30]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
print("query_embedding: ", query_embedding[:10])

query_embedding:  [-0.023803711, -0.028808594, 0.018066406, 0.012268066, -0.009521484, 0.022949219, 0.038085938, 0.033447266, 0.015991211, 0.0064086914]


### Retrieve the most relevant chunks from the vector database

We use cosine similarity to find the most similar chunks

In [31]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities)

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:10]
print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 10 chunks after retrieval: ")
for t in top_chunks_after_retrieval:
    print("== " + t)

similarity scores:  [0.6070260825128435, 0.5140547777123434, 0.42276385151343643, 0.28035129853148966, 0.20180335682083902, 0.25183678122910574, 0.23397640817042245, 0.2673734065011327, 0.21428148439455227, 0.20698629739367122, 0.4150179771636488, 0.538982626941801, 0.6267072584453173, 0.4112358089123656, 0.4762826442445846, 0.4714897693633498, 0.2554095067904586, 0.5817763339823819, 0.5788585497112232, 0.37655487716645936, 0.14689359077625935, 0.35183246804848367, 0.11725405844010406, 0.31685145158260436, 0.28882406044618497, 0.27159075832387597, 0.26903592158523926, 0.26794129674743183, 0.3413542633893929, 0.30656099542756354, 0.30766449569756626, 0.3502721685581503, 0.2936397021231952, 0.1912671991188178, 0.30932880683142283, 0.41415755024626105, 0.39932520587047315, 0.39052764095992465, 0.31656485158504816, 0.4305749659813553, 0.34508970777140285, 0.15024401342226973, 0.39083739804819345, 0.1786214943259148, 0.4270237616027157, 0.38720173099104577, 0.1740796633933721, 0.37211344701

## Step 2 - Rerank the chunks retrieved from the vector database

We rerank the 10 chunks retrieved from the vector database. Reranking boosts retrieval accuracy.

Reranking lets us go from 10 chunks retrieved from the vector database, to the 3 most relevant chunks.

In [32]:
response = co.rerank(
    query=query,
    documents=top_chunks_after_retrieval,
    top_n=3,
    model="rerank-v3.5",
)

top_chunks_after_rerank = [top_chunks_after_retrieval[result.index] for result in response.results]
print("Here are the top 3 chunks after rerank: ")
for t in top_chunks_after_rerank:
    print("== " + t)

Here are the top 3 chunks after rerank: 
== Dune: Part Two is a 2024 American epic space opera film directed by Denis Villeneuve, who co-wrote the screenplay with Jon Spaihts. The sequel to Dune (2021), it is the second of a two-part adaptation of the 1965 novel Dune by Frank Herbert. It follows Paul Atreides as he unites with the Fremen people of the desert planet Arrakis to wage war against House Harkonnen. Timothée Chalamet, Zendaya, Rebecca Ferguson, Josh Brolin, Stellan Skarsgård, Dave Bautista, Charlotte Rampling, and Javier Bardem reprise their roles from the first film, with Austin Butler, Florence Pugh, Christopher Walken, and Léa Seydoux joining the cast.
== Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers and Kevin J. Anderson as creative consultant. Legendary CEO Joshua Grod

## Step 3 - Generate the model final answer, given the retrieved and reranked chunks

In [33]:
# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [34]:
# Define documents for RAG
# Each document is passed as context to the model
documents = [
    {"data": {"text": top_chunks_after_rerank[0], "title": "chunk 0"}},
    {"data": {"text": top_chunks_after_rerank[1], "title": "chunk 1"}},
    {"data": {"text": top_chunks_after_rerank[2], "title": "chunk 2"}},
]

# Generate the response using RAG
response = co.chat(
    model="command-a-03-2025",
    messages=[
        {"role": "system", "content": preamble},
        {"role": "user", "content": query}
    ],
    documents=documents,
    temperature=0.3
)

print("Final answer:")
print(response.message.content[0].text)

Final answer:
Denis Villeneuve directed and co-wrote the script for *Dune: Part Two* with Jon Spaihts.

Villeneuve, Mary Parent, and Cale Boyter produced the film, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers and Kevin J. Anderson as creative consultant.


Note: this is indeed the answer you'd expect, and here was the passage of text in wikipedia explaining it!

" [...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

## Bonus: Citations come for free with Cohere! 🎉

At Cohere, all RAG calls come with... precise citations! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.


In [35]:
print("Citations that support the final answer:")
for cite in response.message.citations:
  print(cite)

Citations that support the final answer:
start=0 end=16 text='Denis Villeneuve' sources=[DocumentSource(type='document', id='doc:0', document={'id': 'doc:0', 'text': 'Dune: Part Two is a 2024 American epic space opera film directed by Denis Villeneuve, who co-wrote the screenplay with Jon Spaihts. The sequel to Dune (2021), it is the second of a two-part adaptation of the 1965 novel Dune by Frank Herbert. It follows Paul Atreides as he unites with the Fremen people of the desert planet Arrakis to wage war against House Harkonnen. Timothée Chalamet, Zendaya, Rebecca Ferguson, Josh Brolin, Stellan Skarsgård, Dave Bautista, Charlotte Rampling, and Javier Bardem reprise their roles from the first film, with Austin Butler, Florence Pugh, Christopher Walken, and Léa Seydoux joining the cast.', 'title': 'chunk 0'}), DocumentSource(type='document', id='doc:1', document={'id': 'doc:1', 'text': 'Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian H

In [36]:
def insert_citations_in_order(text, citations):
    """
    A helper function to pretty print citations.
    """
    offset = 0
    document_id_to_number = {}
    citation_number = 0
    modified_citations = []

    # Process citations, assigning numbers based on unique document sources
    for citation in citations:
        citation_numbers = []
        for source in citation.sources:
            doc_id = source.id
            if doc_id not in document_id_to_number:
                citation_number += 1  # Increment for a new document_id
                document_id_to_number[doc_id] = citation_number
            citation_numbers.append(document_id_to_number[doc_id])

        # Adjust start/end with offset
        start, end = citation.start + offset, citation.end + offset
        placeholder = ''.join([f'[{number}]' for number in citation_numbers])
        # Bold the cited text and append the placeholder
        modification = f'**{text[start:end]}**{placeholder}'
        # Replace the cited text with its bolded version + placeholder
        text = text[:start] + modification + text[end:]
        # Update the offset for subsequent replacements
        offset += len(modification) - (end - start)

    # Prepare citations for listing at the bottom, ensuring unique document_ids are listed once
    unique_citations = {doc_id: number for doc_id, number in document_id_to_number.items()}
    citation_list = '\n'.join([f'[{number}] source: "{documents[int(doc_id.split(":")[-1])]["data"]["text"][:100]}..."' 
                              for doc_id, number in sorted(unique_citations.items(), key=lambda item: item[1])])
    text_with_citations = f'{text}\n\n{citation_list}'

    return text_with_citations


print(insert_citations_in_order(response.message.content[0].text, response.message.citations))


**Denis Villeneuve**[1][2][3] **directed**[1][3] and **co-wrote**[1] the script for *Dune: Part Two* with **Jon Spaihts.**[1][2]

**Villeneuve**[2], **Mary Parent**[2][3], and **Cale Boyter**[2] **produced**[2] the film, with **Tanya Lapointe**[2], **Brian Herbert**[2], **Byron Merritt**[2], **Kim Herbert**[2], **Thomas Tull**[2], **Jon Spaihts**[2], **Richard P. Rubinstein**[2], **John Harrison**[2], and **Herbert W. Gain**[2] serving as **executive producers**[2] and **Kevin J. Anderson**[2] as **creative consultant.**[2]

[1] source: "Dune: Part Two is a 2024 American epic space opera film directed by Denis Villeneuve, who co-wrote t..."
[2] source: "Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian ..."
[3] source: "In November 2016, Legendary Pictures obtained the film and TV rights for the Dune franchise, based o..."
